**Rewrite the ICRF3 S/X catalog into .src format**

The ICRF3 S/X catalog downloaded from the [IERS ICRS Center](http://hpiers.obspm.fr/icrs-pc/newwww/index.php
) could not be used directly as the a priori file of radio source position. 
So it needs to be converted to .src format.

Two functions need to be done in this notebook

- **write_solve_src** 

- **write_nnr_list**


# 1) **write_solve_src**

The file Sebastien used for the a priori file of ICRF2 and ICRF3 are listed below.

*icrf2.src*
>    0000+212   0  3 19.35003511    21 29  44.5075377     1.254 


*icrf3.src*
>0000+212     00 03 19.35000596    +21 29 44.5083164    1.0000

The format is slight different.

The column could be represented by the following

- source name (IERS, IVS, or ICRF designation)
- hour, minute, and second of RA
- degree, arcminute, and arcsecond of declination
- formal uncertainty in declination in mas. It will be used as the weight in the NNR if choosing the weighted NNR constraint. And maybe Sebastien prefers the uniform NNR so this column is set as 1.0000.
- comment (optional)

In [1]:
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np
import sys

Then the function is defined below.

In [2]:
def write_sou_src(cat, ofile=None, header="", comments=""):
    """Write source position into .src format

    Parameters
    ----------
    cat: astropy.table.Table object
        catalog
    ofile: string
        output file name
    comments: string
        add comments to the end of each line

    Return
    ------
    None
    """

    # Open the file
    if ofile is None:
        fout = sys.stdout
    else:
        try:
            fout = open(ofile, "w")
        except Exception as e:
            exit()

    # Print header information
    print(header, file=fout)

    # Format of every line
    if len(comments):
        linefmt = "%8s     %02d %02d %11.8f    %s %02d %10.7f  %7.3f  ! %s"
    else:
        linefmt = "%8s     %02d %02d %11.8f    %s %02d %10.7f  %7.3f"

    # Loop to write the position for every source
    for i in range(len(cat)):
        souname = cat[i]["source_name"]
        ra, dec = cat[i]["ra"], cat[i]["dec"]
        dec_err = cat[i]["dec_err"]

        raang = Angle(ra * u.deg)
        decang = Angle(dec * u.deg)
        rah, ram, ras = raang.hms
        decd, decm, decs = decang.dms

        # When converting an Angle object into degrees using dms method,
        # the degree, arcminute, and arcsecond extracted from the resulted tuple
        # would be all negative if the Angle is negative.
        # But we only want the degree part to be negative.
        if dec < 0 and decd == 0:
            decdstr = "-00"
        else:
            decdstr = "{:+03.0f}".format(decd)

        decm, decs = np.fabs(decm), np.fabs(decs)

        if len(comments):
            print(linefmt % (souname, rah, ram, ras, decdstr, decm, decs, dec_err,
                             comments), file=fout)
        else:
            print(linefmt % (souname, rah, ram, ras, decdstr, decm, decs, dec_err), file=fout)

    # Close the file
    if fout is not sys.stdout:
        fout.close()

I perform a test the code on  the ICRF2 and ICRF3 catalogs.

In [3]:
from read_icrf import read_icrf2

icrf2 = read_icrf2()
icrf2.rename_column("iers_name", "source_name")
icrf2sub = icrf2[icrf2["source_name"] == "0000+212"]
write_sou_src(icrf2sub)


0000+212     00 03 19.35003510    +21 29 44.5075377    1.252


In [4]:
from read_icrf import read_icrf3

icrf3 = read_icrf3()
icrf3.rename_column("iers_name", "source_name")
icrf3sub = icrf3[icrf3["source_name"] == "0000+212"]
write_sou_src(icrf3sub)


0000+212     00 03 19.35000596    +21 29 44.5083164    0.173


It works.

In [5]:
write_sou_src(icrf3, "../logs/icrf3-sx.src")

# 2) **write_nnr_list**

This is very simple, just to print the source list by every 8 sources per line.

In [6]:
def write_nnr_list(soulist, ofile=None):
    """Write source list by every 8 sources per line

    Parameters
    ----------
    soulist: astropy.table.Table object
        source list
    ofile: string
        output file name

    Return
    ------
    None
    """

    # Open the file
    if ofile is None:
        fout = sys.stdout
    else:
        fout = open(ofile, "w")

    # Loop to write 
    print("    ", file=fout, end="")
    for i, sou in enumerate(soulist):
        print("{:8}  ".format(sou), file=fout, end="")
        
        if not (i+1) % 8:
            print("\\\n    ", file=fout, end="")

    # Close the file
    if fout is not sys.stdout:
        fout.close()

For ICRF2 defining sources,

In [7]:
icrf2def = icrf2[icrf2["type"] == "D"]

write_nnr_list(icrf2def["source_name"])

    0002-478  0007+106  0008-264  0010+405  0013-005  0016+731  0019+058  0035+413  \
    0048-097  0048-427  0059+581  0104-408  0107-610  0109+224  0110+495  0116-219  \
    0119+115  0131-522  0133+476  0134+311  0138-097  0151+474  0159+723  0202+319  \
    0215+015  0221+067  0230-790  0229+131  0234-301  0235-618  0234+285  0237-027  \
    0300+470  0302-623  0302+625  0306+102  0308-611  0307+380  0309+411  0322+222  \
    0332-403  0334-546  0342+147  0346-279  0358+210  0402-362  0403-132  0405-385  \
    0414-189  0420-014  0422+004  0426+273  0430+289  0437-454  0440+345  0446+112  \
    0454-810  0454-234  0458-020  0458+138  0506-612  0454+844  0506+101  0507+179  \
    0516-621  0515+208  0522-611  0524-460  0524-485  0524+034  0529+483  0534-611  \
    0534-340  0537-441  0536+145  0537-286  0544+273  0549-575  0552+398  0556+238  \
    0600+177  0642+449  0646-306  0648-165  0656+082  0657+172  0707+476  0716+714  \
    0722+145  0718+792  0727-115  0736+017  0738+491  

For ICRF3 defining sources,

In [10]:
icrf3def = icrf3[icrf3["type"] == "D"]

write_nnr_list(icrf3def["source_name"])

icrf3def.write("../logs/icrf3sx-defsou.list", format="ascii.csv",
              include_names=["source_name"],
              overwrite=True)

    0002-478  0007+106  0009-148  0010+405  0013-005  0017+200  0016+731  0035-252  \
    0038-326  0044-846  0046+316  0047+023  0048-097  0054+161  0059+581  0104-408  \
    0107-610  0110+495  0133+476  0149+218  0159+723  0202-172  0202+319  0208-512  \
    0215+015  0221+067  0227-369  0230-790  0227+403  0234-301  0235-618  0235+164  \
    0237-027  0239+175  0256-005  0300+470  0302+625  0305+039  0308-611  0307+380  \
    0312+100  0316-444  0322+222  0332-403  0334-131  0346-279  0347-211  0346+800  \
    0355-669  0400-319  0402-362  0403-132  0406-127  0415+398  0420+022  0430+289  \
    0437-454  0454-810  0454-234  0458-020  0506-612  0454+844  0506+101  0507+179  \
    0510+559  0515+208  0522-611  0524-485  0524+034  0530-727  0529+483  0534-340  \
    0536+145  0537-286  0539-057  0544+273  0548+084  0552+398  0556+238  0605-085  \
    0607-157  0613+570  0615+820  0627-199  0642-349  0641+392  0646-306  0648-165  \
    0657+172  0700-465  0700-197  0716+714  0727-115  